<a href="https://colab.research.google.com/github/rania-alhabsyi/MachineLearningTasks/blob/main/TaskWeek7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The model is retrieved from

https://huggingface.co/mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis/tree/main

The lines of code below are mostly generated with the help of ChatGPT.

In [1]:
# Step 1: Set up Google Colab
# First, install Hugging Face's Transformers library and PyTorch if they are not already installed
!pip install transformers torch

# Step 2: Import required libraries
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import numpy as np
from sklearn.metrics import accuracy_score

# Step 3: Load the model and tokenizer
model_name = "mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Step 4: Define a function to predict sentiment
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
    sentiment = torch.argmax(probs).item()
    return sentiment, probs.tolist()[0]

# Test with a sample sentence
sentence = "The company is expected to have a strong financial quarter."
sentiment, probabilities = predict_sentiment(sentence)
print(f"Sentiment: {sentiment}, Probabilities: {probabilities}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/933 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/328M [00:00<?, ?B/s]

Sentiment: 2, Probabilities: [0.00030763354152441025, 0.00016340102592948824, 0.9995290040969849]


In [2]:
# Step 1: Define a synthetic dataset
synthetic_data = [
    ("The stock market is booming", 1),   # Positive
    ("The company faces legal issues", 0),  # Negative
    ("Profits are at an all-time high", 1), # Positive
    ("There are concerns about bankruptcy", 0), # Negative
    ("The new product was well received", 1), # Positive
    ("The sales have dropped significantly", 0) # Negative
]

# Step 2: Predict sentiments for the synthetic dataset
y_true = []
y_pred = []
for sentence, label in synthetic_data:
    sentiment, _ = predict_sentiment(sentence)
    y_true.append(label)
    y_pred.append(sentiment)

# Step 3: Calculate accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f"Model accuracy on synthetic data: {accuracy:.2f}")

Model accuracy on synthetic data: 0.17


In [5]:
from torch import nn

class EnhancedDistilRoBERTa(nn.Module):
    def __init__(self, base_model):
        super(EnhancedDistilRoBERTa, self).__init__()
        self.base_model = base_model
        self.attention = nn.MultiheadAttention(embed_dim=768, num_heads=8)
        self.classifier = nn.Linear(768, 2)  # Assuming binary classification

    def forward(self, input_ids, attention_mask=None):
        # Pass `output_hidden_states=True` to get hidden states
        outputs = self.base_model(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True)
        # Access the last hidden state from the hidden_states tuple
        last_hidden_state = outputs.hidden_states[-1]
        attn_output, _ = self.attention(last_hidden_state, last_hidden_state, last_hidden_state)
        pooled_output = attn_output.mean(dim=1)
        logits = self.classifier(pooled_output)
        return logits

# Initialize the enhanced model with attention
enhanced_model = EnhancedDistilRoBERTa(model)

In [6]:
# Define a function to predict with the enhanced model
def predict_sentiment_enhanced(text):
    inputs = tokenizer(text, return_tensors="pt")
    logits = enhanced_model(**inputs)
    probs = torch.nn.functional.softmax(logits, dim=-1)
    sentiment = torch.argmax(probs).item()
    return sentiment, probs.tolist()[0]

# Evaluate enhanced model on synthetic data
y_pred_enhanced = []
for sentence, label in synthetic_data:
    sentiment, _ = predict_sentiment_enhanced(sentence)
    y_pred_enhanced.append(sentiment)

# Calculate accuracy for the enhanced model
accuracy_enhanced = accuracy_score(y_true, y_pred_enhanced)
print(f"Enhanced model accuracy on synthetic data: {accuracy_enhanced:.2f}")

# Compare the results
print(f"Original model accuracy: {accuracy:.2f}")
print(f"Enhanced model accuracy with attention: {accuracy_enhanced:.2f}")

Enhanced model accuracy on synthetic data: 1.00
Original model accuracy: 0.17
Enhanced model accuracy with attention: 1.00


Model's accuracy is better when the model was enhanced.